In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import datetime
from pandas.io.json import json_normalize
import gmaps
import gmaps.datasets
from config import gkey

ModuleNotFoundError: No module named 'config'

In [ ]:
# –––––––––––––––––––––––––– * API CALLS * ––––––––––––––––––––––––––  #
#Retrieve the data

start_date = datetime.date(year=2017, month=9, day=1)
end_date = datetime.date(year=2018, month=12, day=1)
period = datetime.timedelta(days=1)

data = []

date = start_date

while date < end_date:
    the_date = date.strftime('%Y-%m-%d')
    url = f"https://data.austintexas.gov/resource/r3af-2r8x.json?$$app_token=PueZjmFITn1jWUQCgPpRj0QkW&$where=published_date between \'{the_date}T00:00:00\' and \'{the_date}T23:59:59\'"
    response = requests.get(url).json()
    data += response
    date += period
    
data

In [ ]:
# Converts the Data into a DataFrame
traffic_df = pd.DataFrame.from_dict(json_normalize(data), orient='columns')

# Save data as csv
traffic_df.to_csv('Resources/traffic_raw.csv')

traffic_df.head()

In [ ]:
# See if there is any missing values on the data acquired
traffic_df.count()

In [ ]:
# Drop incomplete values
traffic_df=traffic_df.dropna()
traffic_df.count()

In [ ]:
# Find the diferent categories for "issue reported"
traffic_df['issue_reported'].value_counts()

In [ ]:
# –––––––––––––––––––––––––– * RENAMING CATEGORIES * ––––––––––––––––––––––––––  #
# Clean up issue_reported category. Replace 'Accident/Collision' in a single category.

#ACCIDENT COLLISION
traffic_df['issue_reported'] = traffic_df['issue_reported'].replace({
    'Crash Urgent': 'Accident/Collision', 'Crash Service': 'Accident/Collision',
    'COLLISION': 'Accident/Collision', 'COLLISION WITH INJURY': 'Accident/Collision',
    'COLLISN/ LVNG SCN': 'Accident/Collision', 'COLLISN / FTSRA': 'Accident/Collision',
    'COLLISION/PRIVATE PROPERTY': 'Accident/Collision', 'COLLISN/ LVNG SCN': 'Accident/Collision',
    'AUTO/ PED': 'Accident/Collision','FLEET ACC/ INJURY': 'Accident/Collision',
    'TRAFFIC FATALITY': 'Accident/Collision'})

#TRAFFIC IMPEDIMENT
traffic_df['issue_reported'] = traffic_df['issue_reported'].replace({
    'zSTALLED VEHICLE': 'Traffic Impediment', 'BLOCKED DRIV/ HWY': 'Traffic Impediment',
    'LOOSE LIVESTOCK': 'Traffic Impediment'})


#TRAFFIC HAZARD
traffic_df['issue_reported'] = traffic_df['issue_reported'].replace({
    'TRFC HAZD/ DEBRIS': 'Traffic Hazard', 'N / HZRD TRFC VIOL': 'Traffic Hazard',
    'ICY ROADWAY': 'Traffic Hazard', 'HIGH WATER': 'Traffic Hazard',
    'VEHICLE FIRE': 'Traffic Hazard', 'HIGH WATER': 'Traffic Hazard'})

# Drop the values for 'BOAT ACCIDENT' since those are not of interest for the study.
traffic_df = traffic_df[traffic_df.issue_reported != 'BOAT ACCIDENT']

traffic_df['issue_reported'].value_counts()

In [ ]:
traffic_df.to_csv('Resources/traffic (main).csv') #clean dataset to work with

In [ ]:
traffic_df.head()

In [2]:
#retrieve lat/lng values using numpy
gmaps.configure(api_key= gkey)
fig = gmaps.figure()
austin_coords = (30.26, -97.74)
fig = gmaps.figure(center=austin_coords, zoom_level=12)
gmaps.figure(map_type='ROADMAP')
fig

NameError: name 'gkey' is not defined